In [2]:
import boto3
import pandas as pd
import librosa
import io
from io import BytesIO
from IPython.display import Audio

In [3]:
#Loading the data into the notebook
s3 = boto3.client('s3')
bucket_name = '2307-01-acoustic-loggers-for-leak-detection-a'
object_key = 'Metadata_Audio_Connected/Connected_data.xlsx'

response = s3.get_object(Bucket=bucket_name, Key=object_key)
excel_data = response['Body'].read()

df = pd.read_excel(BytesIO(excel_data))
print('Connected Loaded Successfully into the notebook')

Connected Loaded Successfully into the notebook


In [4]:
df.head()

,datetime,siteid,recording_id,file,postcodedistrict,dmacode,leak_found,noise,spread,repaired_as,Audio_key
0,2018-12-12 04:00:00,1668120,26577010,../recordings/2018/12/12/recordings_1668120_26...,NW10,ZSUHIL25,No,15.0,4.0,NaN,Unstructured audio files/2018/12/12/recordings...
1,2018-12-12 04:00:00,1742872,26592074,../recordings/2018/12/12/recordings_1742872_26...,E15,ZWOODF113,No,22.0,9.0,NaN,Unstructured audio files/2018/12/12/recordings...
2,2018-12-12 04:00:00,1616760,26593071,../recordings/2018/12/12/recordings_1616760_26...,HP12,ZWIDDN02,No,21.0,7.0,NaN,Unstructured audio files/2018/12/12/recordings...
3,2018-12-12 04:00:00,1630929,26593758,../recordings/2018/12/12/recordings_1630929_26...,SL1,ZSTKWD30,No,14.0,5.0,NaN,Unstructured audio files/2018/12/12/recordings...
4,2018-12-12 04:00:00,6896951,26596303,../recordings/2018/12/12/recordings_6896951_26...,SL7,ZMARLC01,No,10.0,3.0,NaN,Unstructured audio files/2018/12/12/recordings...


In [5]:
def normalize_audio(audio_data):
    normalized = librosa.util.normalize(audio_data)
    return normalized

In [6]:
import concurrent.futures

In [47]:
def process_audio_r(i):
    try:
        bucket_name = '2307-01-acoustic-loggers-for-leak-detection-a'
        audio_file_key = df.loc[i, 'Audio_key']

        # Download audio file from S3
        response = s3.get_object(Bucket=bucket_name, Key=audio_file_key)
        audio_data = response['Body'].read()

        # Load audio data and sampling rate
        signal, sr = librosa.load(io.BytesIO(audio_data))

        # Normalize audio data
        normalized_audio = librosa.util.normalize(signal)

        return normalized_audio

    except Exception as e:
        print(f"Error processing audio {i}: {e}")
        return None

if __name__ == "__main__":
    # Initialize S3 client
    s3 = boto3.client('s3')
    
    # Initialize lists to store DataFrames
    df_list_r = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_audio_r, i) for i in range(100)]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            df_list_r.append(result)

    # Concatenate DataFrames into a single DataFrame
     
    df_Normalized = pd.DataFrame({'Normalized': df_list_r})

In [48]:
audio_data = df_Normalized.loc[0, 'Normalized']

In [18]:
from scipy.io import wavfile

In [40]:
bucket_name = '2307-01-acoustic-loggers-for-leak-detection-a'
audio_file_key= df.loc[0, 'Audio_key']  # Path to the audio file in the S3 bucket

# Download the audio file from S3
response = s3.get_object(Bucket=bucket_name, Key=audio_file_key)
audio_data_Leak = response['Body'].read()

# Display the audio file using the IPython Audio widget
audio = Audio(audio_data_Leak)

In [29]:
# Initialize the S3 client
s3 = boto3.client('s3')

# Define your S3 bucket name and audio file key
bucket_name = '2307-01-acoustic-loggers-for-leak-detection-a'

# Create a file-like object to store the audio data
audio_data = io.BytesIO()

# Stream the audio file from S3 into the file-like object
s3.download_fileobj(bucket_name, audio_file_key, audio_data)

# Reset the file-like object's position to the beginning
audio_data.seek(0)

# Load the audio data using Librosa
y, sr = librosa.load(audio_data)

In [36]:
sr

22050

In [31]:
sampling_rate = 22050

In [49]:
wavfile.write('restored_audio.wav', audio_data)

TypeError: write() missing 1 required positional argument: 'data'

In [50]:
librosa.output.write_wav(audio_data, sr=sampling_rate, norm=True)

AttributeError: No librosa attribute output

In [52]:
!pip install noisereduce
import noisereduce as nr
import soundfile as sf

# Load audio file
audio_file = 'input_audio.wav'
data, sr = sf.read(audio_data)

# Reduce noise
reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=data[:sr*5], verbose=False)

# Save processed audio
sf.write('output_audio.wav', reduced_noise, sr)


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


TypeError: Invalid file: array([0.5432981 , 0.62693447, 0.65527993, ..., 0.2732673 , 0.17232941,
       0.08091287], dtype=float32)